### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from functools import reduce

# Raw data file
file_to_load = "Resources/purchase_data.csv"

# Read purchasing file and store into pandas data frame
purchase_data = pd.read_csv(file_to_load)

#look at duplicate SN data
duplicateInfo = purchase_data[purchase_data.duplicated(['SN'],keep=False)].sort_values(by=['SN'])
duplicateInfo.head()


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
218,218,Aelastirin39,23,Male,76,Haunted Bronzed Bludgeon,3.15
766,766,Aelastirin39,23,Male,58,"Freak's Bite, Favor of Holy Might",4.14
584,584,Aelin32,20,Male,115,Spectral Diamond Doomblade,2.04
87,87,Aelin32,20,Male,151,Severance,3.40
52,52,Aelin32,20,Male,60,Wolf,3.54


## Player Count

* Display the total number of players


In [3]:
#Get total number of players and total number of paying players
totalPlayers=1163
totPayingPlayers = purchase_data.SN.nunique()

#print("Total number of players: " +str(totalPlayers)+ " (Given in the information at the beginning)")
print("Total number of players who made at least one purchase: "+str(totPayingPlayers))


Total number of players who made at least one purchase: 576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
#Run basic calculations to obtain number of unique items, average price of an item, total number of purchases, total revenue
numUniItems = purchase_data["Item ID"].nunique()
avePrice = purchase_data["Price"].mean()
numPurchases = purchase_data["Purchase ID"].count()
totRev = purchase_data["Price"].sum()

#Create a summary data frame to hold the results.
summaryData = {"Number of Unique Items": [numUniItems], "Average Price": ["${:,.2f}".format(avePrice)],
               "Number of Purchases": [numPurchases], "Total Revenue": ["${:,.2f}".format(totRev)]}
summaryData_df = pd.DataFrame(data=summaryData)
summaryData_df


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [5]:
#Get list of genders in the data
genders = purchase_data.Gender.unique()

#Given information from above on percentage of total number of players by gender in the game
percentNumberPlayers = {"Male":.84, "Female":.14, "Other / Non-Disclosed":(1-.84-.14)}

#create list of lists to be used to create dataframe of gender information
genderInfo = [[gender, 
               "{:0.0f}".format(percentNumberPlayers.get(gender)*totalPlayers),  
               purchase_data.loc[purchase_data["Gender"]==gender]["SN"].nunique(),
               "{:0.0f}%".format(percentNumberPlayers.get(gender)*100),
               "{:0.2f}%".format((purchase_data.loc[purchase_data["Gender"]==gender]["SN"].nunique()/totPayingPlayers)*100)] 
              for gender in genders]

#convert list of lists to dataframe of gender info and name columns
genderInfo_df = pd.DataFrame(genderInfo, columns=["Gender",
                                                  "Count of Players",
                                                  "Count of Paying Players",
                                                  "Percentage of Total Players",
                                                  "Percentage of Paying Players"])
#Set Gender column as index column
genderInfo_df.set_index("Gender", inplace=True)

#Sort descending by count
genderInfo_df = genderInfo_df.sort_values(by=["Count of Paying Players"], ascending = False)


#Show dataframe dropping unwanted columns
genderInfo_df.drop(columns=["Count of Players", "Percentage of Total Players"])


,Count of Paying Players,Percentage of Paying Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [6]:
#Get total purhcase value (and format the column) and total number of purchases for each gender and insert as new column
genderTotalPurchValue = [] 
genderNumPurchases = []

for gender in genderInfo_df.index:
    genderTotalPurchValue.append(purchase_data.loc[purchase_data["Gender"]== gender]["Price"].sum())
    genderNumPurchases.append(purchase_data.loc[purchase_data["Gender"]== gender]["SN"].count())


genderInfo_df["Total Number of Purchases"] = genderNumPurchases
genderInfo_df["Total Purchase Value"] = genderTotalPurchValue


#Get average purchase price and average total purchases value per player and insert as new column with correct formating
genderInfo_df["Average Purchase Price"]=(genderInfo_df["Total Purchase Value"]/genderInfo_df["Total Number of Purchases"]).map("${:,.2f}".format)
genderInfo_df["Average Total Purchase Value per Paying Player"]=(genderInfo_df["Total Purchase Value"]/genderInfo_df["Count of Paying Players"]).map("${:,.2f}".format)



genderInfo_df["Total Purchase Value"] = genderInfo_df["Total Purchase Value"].map("${:,.2f}".format)
 
genderInfo_df
genderInfo_df.drop(columns=["Count of Players", "Count of Paying Players",
                            "Percentage of Total Players","Percentage of Paying Players"])


,Total Number of Purchases,Total Purchase Value,Average Purchase Price,Average Total Purchase Value per Paying Player
Gender,,,,
Male,652,"$1,967.64",$3.02,$4.07
Female,113,$361.94,$3.20,$4.47
Other / Non-Disclosed,15,$50.19,$3.35,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [7]:
# Establish bins for ages
age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#Find Min and Max Ages
minAge=purchase_data["Age"].min()
maxAge=purchase_data["Age"].max()

#Include only one purchase per SN
no_duplicates=purchase_data.drop_duplicates(["SN"])
no_duplicates["Age Group"] = pd.cut(no_duplicates["Age"], age_bins, labels=group_names)

#Create Summary Data Frame
age_df =  no_duplicates.groupby(["Age Group"])[["SN"]].count()
age_df.columns = ["Count of Paying Players"]
age_df["Percentage of Paying Players"]=((age_df["Count of Paying Players"]/totPayingPlayers)*100).map("{:.2f}%".format)

print(f"The youngest paying player is {minAge} years old.")
print(f"The oldest paying player is {maxAge} years old.")
age_df

#The following will produce the output in the example, but I think it counts users who made 2 or more purchases multiple times
#purchase_data["Age Group"] = pd.cut(purchase_data["Age"], age_bins, labels=group_names)

#age_df =  purchase_data.groupby(["Age Group"])[["SN"]].count()
#age_df.columns = ["Count of Paying Players"]
#age_df["Percentage of Paying Players"]=((age_df["Count of Paying Players"]/totPayingPlayers)*100).map("{:.2f}%".format)
#age_df


The youngest paying player is 7 years old.
The oldest paying player is 45 years old.


C:\Users\rkalh\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,Count of Paying Players,Percentage of Paying Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [8]:
purchase_data["Age Group"] = pd.cut(purchase_data["Age"], age_bins, labels=group_names)

dfs = [age_df.drop(columns=["Percentage of Paying Players"]),purchase_data.groupby(["Age Group"])[["Purchase ID"]].count(),
                       purchase_data.groupby(["Age Group"])[["Price"]].sum()]

#Create Summary Data Frame
purchase_df = reduce(lambda left,right: pd.merge(left,right,on="Age Group"), dfs)
purchase_df = purchase_df.rename({"Purchase ID":"Purchase Count","Price":"Total Purchase Value"},axis="columns")
purchase_df["Average Purchase Price"]=(purchase_df["Total Purchase Value"]/purchase_df["Purchase Count"]).map("${:.2f}".format)

#Does not match output from example becuase I used count of unique screen names, otherwise Average Purchase Price and
#Average Purchase Total Per Person are the same
purchase_df["Average Purchase Total Per Paying Player"]=(purchase_df["Total Purchase Value"]/purchase_df["Count of Paying Players"]).map("${:.2f}".format)

purchase_df["Total Purchase Value"]=purchase_df["Total Purchase Value"].map("${:,.2f}".format)


purchase_df.drop(columns=["Count of Paying Players"])


,Purchase Count,Total Purchase Value,Average Purchase Price,Average Purchase Total Per Paying Player
Age Group,,,,
<10,23,$77.13,$3.35,$4.54
10-14,28,$82.78,$2.96,$3.76
15-19,136,$412.89,$3.04,$3.86
20-24,365,"$1,114.06",$3.05,$4.32
25-29,101,$293.00,$2.90,$3.81
30-34,73,$214.00,$2.93,$4.12
35-39,41,$147.67,$3.60,$4.76
40+,13,$38.24,$2.94,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [12]:
#Identify the top 5 spenders
top_five_data=pd.merge(purchase_data.groupby(["SN"])[["Purchase ID"]].count(),
                       purchase_data.groupby(["SN"])[["Price"]].sum(), on="SN").sort_values(by=['Price'],ascending=False)
top_five_data=top_five_data.rename({"Purchase ID":"Purchase Count","Price":"Total Purchase Value"},axis="columns")
top_five_data["Average Purchase Price"]=(top_five_data["Total Purchase Value"]/top_five_data["Purchase Count"]).map("${:.2f}".format)
top_five_data["Total Purchase Value"]=top_five_data["Total Purchase Value"].map("${:.2f}".format)
top_five_data.head()

,Purchase Count,Total Purchase Value,Average Purchase Price
SN,,,
Lisosia93,5,$18.96,$3.79
Idastidru52,4,$15.45,$3.86
Chamjask73,3,$13.83,$4.61
Iral74,4,$13.62,$3.40
Iskadarya95,3,$13.10,$4.37


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [13]:
#Identify the top 5 items
#Group by Item ID (and Item Name) to get purchase count and total purchase value
#Also, change column names
top_five_items=pd.merge(purchase_data.groupby(["Item ID","Item Name"])[["Purchase ID"]].count(),
                        purchase_data.groupby(["Item ID", "Item Name"])[["Price"]].sum(),
                        on = ["Item ID","Item Name"])
top_five_items=top_five_items.rename({"Purchase ID":"Purchase Count","Price":"Total Purchase"},axis="columns")

#Get item price and sort by Purchase count
top_five_items=pd.merge(top_five_items,
                        purchase_data.groupby(["Item ID", "Item Name"])[["Price"]].min(),
                        on = ["Item ID","Item Name"]).sort_values(by=["Purchase Count"],ascending=False)

#clean up formatting
#keep total purchase as the numerical value to sort on in next part as opposted to total purchase value that is an object
top_five_items["Price"]=(top_five_items["Price"]).map("${:.2f}".format)
top_five_items["Total Purchase Value"]=(top_five_items["Total Purchase"]).map("${:.2f}".format)

#print first 5 entries
top_five_items.drop(columns=["Total Purchase"]).head()


,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [14]:
top_five_items = top_five_items.sort_values(by=["Total Purchase"], ascending=False)
top_five_items.drop(columns=["Total Purchase"]).head()


,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


Observations:
While there are far fewer female paying players than male, their spending on average (both total per player and per item) is higher than their male counterparts. This could suggest that while women in general are less likely to play, the ones that do play are more likely to invest in the playing experience. More investigation into the overall shape and spread of the data per gender would have to be done to verify this.

Of the paying players, the 5 that spent the most or the top 0.7% of spenders, account for 8.8% of the total purchase value. This would suggest that you have a few that pay a lot, relatively speaking and a lot of players that pay a relatively small amount. Again, here a look at the shape of the data would help with this exploration.

In the age data, clearly the 20-24 age group is the biggest in terms of players and total amount purchased by a large amount, but it doesn't seem to be a standout as far as average purchase total or average purchase price.